# Advanced Lane Finding

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
%matplotlib inline

In [ ]:
# Read in calibration images
images = glob.glob('../calibation_images/calibration*.jpg')

# Arrays to store object points and image points from all the images

objpoints = [] # 3D points in a real world image space
impoints = [] # 3D points in an image plane

# Prepare object points (x, y, z)
# Our checkerboard has 6 x 8 points
objp = np.zeros((6*8, 3), np.float32)

# np.mgrid returns the coordinate value for a given grid size
obj[:,:,2] = np.mgrid[0,:,0,6].T.reshape(-1,2) # x,y cordinates

# Iterate through images
for img in images:
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (8,6), None)

    # If corners are found, add object points, image points
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (8,6), corners, ret)
        plt.imshow(img)
    


# Undistort An Image

In [ ]:
for image in images:
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[:,:,-1], None, None)
    dst = cv2.undistort(image, mtx, dist, None, mtx)

# Transform An Image Perspective

In [ ]:
# Compute transform given source and destination points:
M = cv2.getPerspectiveTransform(src,dst)

# Compute the inverse perspective transform:
M_inv = cv2. getPerspectiveTransform(dst, src)

# Warp image using perspective transform:
warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

# Sobel Operator

In [ ]:
# First convert to grayscale because Sobel only accepts one color channel
gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

# Calculate the derivative in the x direction
sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)

# Calculate the derivative in the y direction
sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)

# Convert absolute value of x derivative
abs_sobelx = np.absolute(sobelx)
scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

# Create a binary threshold to select pixels based on gradient strength
thesh_min = 20
thresh_max = 100
sxbinary = np.zeros_like(scaled_sobel)
sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
plt.imshow(sxbinary, cmap='gray')

# Sobel Magnitude

In [ ]:
abs_sobelx = sqrt(sobelx^2)
abs_sobely = sqrt(sobely^2)
abs_sobelxy = sqrt(sobelx^2 + sobely^2)

# Direction of Gradient

In [ ]:
# Direction of the gradient is the inverse tangent of y gradient divided by x gradient
# arctan(sobelx/sobely)
direction = np.arctan2(sobely/sobelx)

# HLS and Color Thresholds

In [ ]:
image = mpimg.imread('test6.jpg')

# Get channels in RGB space
# R = image[:,:,0]
# G = image[:,:,1]
# B = image[:,:,2]

# Get channels in HLS
# hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
# H = hls[:,:,0]
# L = hls[:,:,1]
# S = hls[:,:,2]

thresh = (180, 255)
gray = cvt.Color(image, cv2.COLOR_RGB2GRAY)
binary = np.zeros_like(gray)
binary[(gray > thresh[0]) & (gray <= thresh[1])] = 1

# Color and Gradient

In [ ]:
# Convert to HLS color space and separate the S channel
# using an undistorted image
hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
s_channel = hl[:,:,2]

# Grayscale image
# Grayscaling lost color information for the lane lines previously
# Explore gradients in other color spaces/channels
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Sobel X
sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
abs_sobelx = np.absolute(sobelx)
scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

# Threshold x gradient
thresh_min = 20
thresh_max = 100
sxbinary = np.zeros_like(s_channel)
sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

# Threshold color channel
s_thresh_min = 170
s_thresh_max = 255
s_binary = np.zeros_lik(s_channel)
s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

# Stack each channel to view individual contributions in green and blue respectively
# This returns a stack of two binary images, whose components you can see as different colors
color_binary = np.dstack((np.zeros_like(sxbinary), sxbinary, s_binary))

# Combine the two binary thresholds
combined_binary = np.zeros_like(sxbinary)
combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

# Plotting threshold images
f, (ax1, ax2) = plt.subplots(1, 2, figsize(20, 100))
ax1.set_title('Stakced thresholds')
ax1.imshow(color_binary)

ax.set_title('Combined S channel and gradient thresholds')
ax2.imshow(combined_binary, cmap='gray')



# Locate Lane Lines and Fit A Polynomial

In [ ]:
#  Take histogram along all columns in the lower half of the image
histogram = np.sum(img[img.shape[0]/2:,:], axis=0)
plt.plt(histogram)

# Implement sliding windows and fit a polynomial

histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)

# Create an output image to draw on and visualize the result
out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

# Find the peak of the left and right halves of the historgram
# These will be the starting point for the left and right lines
midpoint = np.int(histogram.shape[0]/2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

# Choose the number of sliding windows
n_windows = 9

# Set height of windows
window_height = np.int(binary_warped.shape[0]/n_windows)

# Identify the x and y positions of all nonzero pixels in the image
nonzero = binary_warped.nonzero()
nonzero_y = np.array(nonzero[0])
nonzero_x = np.array(nonzero[1])


